## 来个例子回顾一下

现在我们有两种 MC 生物，即两种产品，不使用工厂模式的话，每种生物的实例化方法的都要暴露出来。

In [1]:
# 不使用工厂模式

"""有两种生物"""


class Villager:
    """村民"""

    def say(self):
        print("哈↓哈？")


class Zombie:
    """僵尸"""
    
    def say(self):
        print("嗷～")
    

villager = Villager()
villager.say()

zombie = Zombie()
zombie.say()

哈↓哈？
嗷～


## 简单工厂模式

现在回顾一下简单工厂模式，把不同的类的实例化统一到一个工厂里，不对外暴露创建接口。

In [2]:
class SimpleMobEgg:
    """简单生物蛋"""


    """if else 当然可以 work"""
    # def hatch(self, mob_name):
    #     if mob_name == "villager":
    #         return Villager()
    #     elif mob_name == "zombie":
    #         return Zombie()
        
    """也可以用 dictionary"""
    _mob_dict = {
        "villager": Villager,
        "zombie": Zombie,
    }

    def hatch(self, mob_name):
        return self._mob_dict[mob_name]()
        

In [3]:
# 这个工厂可以生成多种产品 -> 统一实例化接口
simple_mob_egg = SimpleMobEgg()

# 这里才确定是哪种生物
villager = simple_mob_egg.hatch(mob_name="villager")
villager.say()

# 同上
zombie = simple_mob_egg.hatch(mob_name="zombie")
zombie.say()

哈↓哈？
嗷～


有什么问题呢？

这里只支持两种生物，要么 if else

```python
    def hatch(self, mob_name):
        if mob_name == "villager":
            return Villager()
        elif mob_name == "zombie":
            return Zombie()
```

要么字典

```python
    _mob_dict = {
        "villager": Villager,
        "zombie": Zombie,
    }

    def hatch(self, mob_name):
        return self._mob_dict[mob_name]()
```

都写死了（悲）

适用场景：已经确定有多少具体的类，不会再多加别的。要加的话就得改 `SimpleMobEgg` 类了，违背了 **开闭原则**

## 开闭原则

> 在面向对象编程领域中，开闭原则 (The Open/Closed Principle, OCP) 规定“软件中的对象（类，模块，函数等等）应该对于扩展是开放的，但是对于修改是封闭的”，这意味着一个实体是允许在不改变它的源代码的前提下变更它的行为。

让系统易于扩展，同时限制其每次被修改所影响的范围。

## 工厂方法模式（Factory method pattern）

- 处理不指定对象具体类型的情况下创建对象的问题
- 定义一个创建对象的接口，但让实现这个接口的类来决定实例化哪个类
- 类的实例化推迟到子类中进行

In [4]:
from abc import ABC, abstractmethod

# 这是真正用于实例化的类，但我们不想暴露
class Villager:
    """村民"""

    def say(self):
        print("哈↓哈？")


class Zombie:
    """僵尸"""
    
    def say(self):
        print("嗷～")


# 抽象工厂：先定义抽象类，然后每种类型的产品都有对应的工厂
class AbstractMobEgg(ABC):
    """抽象工厂"""

    @abstractmethod
    def hatch(self):
        """这里只定义了需要实现的方法，没有实现（接口）"""
        pass


# 对每种产品的工厂
class VillagerEgg(AbstractMobEgg):
    """村民蛋"""

    def hatch(self):
        return Villager()
    
class ZombieEgg(AbstractMobEgg):
    """僵尸蛋"""

    def hatch(self):
        return Zombie()


# 使用
villager_egg = VillagerEgg()    # 实例化村民蛋，这个对象可以孵化出村民
villager = villager_egg.hatch() # 用村民蛋实例化村民
villager.say()

zombie_egg = ZombieEgg()
zombie = zombie_egg.hatch()
zombie.say()

哈↓哈？
嗷～


如果我们还要再添加新的生物呢？

只要实现 **具体用于实例化的类** 和 **对应于该生物的工厂**

比如我们要添加苦力怕（Creeper）

In [5]:
class Creeper():
    """苦力怕"""

    def say(self):
        print("嗞……崩！")

class CreeperEgg(AbstractMobEgg):
    """苦力怕蛋"""

    def hatch(self):
        return Creeper()
    

creeper_egg = CreeperEgg() 
creeper = creeper_egg.hatch()
creeper.say()

嗞……崩！


## 小节

每个工厂只负责生产自己的产品，即每种生物蛋只孵化对应的生物，避免了新添加生物的时候还要修改已经写好的 `SimpleMobEgg` 类。新增加生物只要实现对应的 `Mob` 类和 `MobEgg` 类就行。